In [1]:
import numpy as np

Import phi and phi_test from train and test datasets using NumPy's loadtxt function

In [4]:
# Import phi from train data set
phi = np.loadtxt('train.csv', dtype='float', delimiter=',', skiprows=1,
                 usecols=tuple(range(1, 14)))

In [5]:
# Import phi_test from test data set
phi_test = np.loadtxt('test.csv', dtype='float', delimiter=',',
                      skiprows=1, usecols=tuple(range(1, 14)))

Import y from train dataset using the loadtxt function

In [7]:

y = np.loadtxt('train.csv', dtype='float', delimiter=',', skiprows=1,
               usecols=14, ndmin=2)

Concatenate coloumn of 1s to right of phi and phi_test

In [8]:
phi_test = np.concatenate((phi_test, np.ones((105, 1))), axis=1)
phi = np.concatenate((phi, np.ones((400, 1))), axis=1)

Apply min max scaling on each coloumn of phi and phi_test

In [9]:
for i in range(0, 13):
    col_max = max(phi[:, i])
    col_min = min(phi[:, i])
    phi[:, i] = (phi[:, i] - col_min) / (col_max - col_min)
    phi_test[:, i] = (phi_test[:, i] - col_min) / (col_max - col_min)

Apply log scaling on y

In [10]:
y = np.log(y)

Define a function to calculate change in error function based on phi, w and p norm

In [11]:
def delta_w(p, phi, w):
    if p == 2:
        deltaw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)))
    if p < 2 and p > 1:
        deltaw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)) * np.sign(w))
    return deltaw


Make a dictionary containing filenames as keys and p as values

In [12]:
filenames = {'output.csv': 2.0,
             'output_p1.csv': 1.75,
             'output_p2.csv': 1.5,
             'output_p3.csv': 1.3
             }

For each item in this dictionary:

Set the w to all 0s

In [13]:
for (fname, p) in filenames.items():
    w = np.zeros((14, 1))

Set an appropriate value for lambda(Hyperparameter) and step size

In [14]:
lambd = 0.2
t = 0.00012 #Max step size

Calculate new value of w

In [15]:
w_new = w - t * delta_w(p, phi, w)

Repeat steps until error between consecutive ws is less than threshold

In [16]:
i = 0
while(np.linalg.norm(w_new-w) > 10 ** -10):
        w = w_new
        w_new = w - t * delta_w(p, phi, w)
        i = i + 1

Load values of id from test data file

In [18]:
id_test = np.loadtxt('test.csv', dtype='int', delimiter=',',
                         skiprows=1, usecols=0, ndmin=2)

Calculate y for test data using phi test and applying inverse log

In [19]:
y_test = np.exp(np.dot(phi_test, w_new))

Save the ids and y according to filename from dictionary

In [20]:
np.savetxt(fname, np.concatenate((id_test, y_test), axis=1),
               delimiter=',', fmt=['%d', '%f'], header='ID,MEDV', comments='')